In [1]:
import torch
from flash_attn import flash_attn_func

ImportError: /home/karel1ne/nlp_course/.venv/lib/python3.10/site-packages/flash_attn_2_cuda.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs

In [2]:
q = torch.randn((16, 512, 16, 64), device='cuda')
k = torch.randn((16, 512, 2, 64), device='cuda')
v = torch.randn((16, 512, 2, 64), device='cuda')

In [3]:
flash_attn_func(q, k, v)

RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [1]:
from model import GovnArgs, GovnLM
import torch

In [2]:
args = GovnArgs(vocab_size=32000, max_seq_len=512)
model = GovnLM(args)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", split="train")
tokenizer.pad_token = tokenizer.eos_token

In [4]:
from datasets import load_dataset
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader

dataset = load_dataset("ashaba1in/small_openwebtext")

def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=512,  
        return_tensors="pt"
    )
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

tokenized_dataset = tokenized_dataset.with_format('torch')

/home/karel1ne/nlp_course/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [48]:
tokenized_dataset = load_dataset('tokenized_smallwebtext').with_format('torch')

/home/karel1ne/nlp_course/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
dataloader = DataLoader(
    tokenized_dataset['train'],
    batch_size=args.max_batch_size,
    shuffle=True
)

In [6]:
batch = next(iter(dataloader))
with torch.no_grad():
    print(batch['input_ids'].shape)
    logits = model(batch['input_ids'])

torch.Size([32, 512])


In [38]:
dataset['train'][0]['input_ids']

[1,
 4194,
 28733,
 581,
 28733,
 3393,
 1261,
 28725,
 382,
 1022,
 28710,
 325,
 28743,
 11348,
 28731,
 1939,
 8599,
 364,
 621,
 13980,
 28725,
 1425,
 372,
 288,
 297,
 3358,
 304,
 26517,
 2917,
 438,
 1411,
 28725,
 6758,
 13500,
 304,
 26579,
 2338,
 1753,
 477,
 264,
 1834,
 6556,
 8085,
 2125,
 1024,
 264,
 12172,
 753,
 5714,
 1918,
 25469,
 25473,
 272,
 2698,
 28725,
 3677,
 378,
 403,
 9059,
 684,
 4908,
 28723,
 13,
 13,
 1014,
 5161,
 1749,
 26656,
 11762,
 12195,
 3198,
 27092,
 308,
 3652,
 28768,
 339,
 2480,
 447,
 28708,
 390,
 272,
 865,
 6676,
 438,
 272,
 6556,
 298,
 625,
 272,
 6883,
 1059,
 272,
 2125,
 28723,
 13,
 13,
 28743,
 11348,
 12735,
 5745,
 28725,
 2818,
 356,
 17256,
 395,
 741,
 302,
 272,
 13500,
 28725,
 369,
 272,
 2969,
 16419,
 8346,
 272,
 12172,
 753,
 4205,
 330,
 313,
 304,
 10731,
 6867,
 298,
 25469,
 11931,
 28723,
 2993,
 28725,
 12172,
 753,
 11762,
 23184,
 10049,
 2404,
 930,
 420,
 1230,
 28713,
 28725,
 264,
 6676,
 693,
 403,
 

In [24]:
tokenized_dataset.save_to_disk('tokenized_smallwebtext')


Saving the dataset (0/14 shards):   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [25]:
from model import GovnArgs, GovnLM
import torch
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import os
from torch import nn
from tqdm.auto import tqdm

In [ ]:
torch.manual_seed(54)
device = 'cuda:0'
args = GovnArgs(
    dim=1024,
    n_layers=36,
    n_heads=16,
    vocab_size=32000,
    feed_forward_hidden_dim=4096,
    norm_eps=1e-6,
    max_batch_size=16,
    max_seq_len=1024
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", split="train")
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset('tokenized_smallwebtext').with_format('torch')
dataloader = DataLoader(
    dataset['train'],
    batch_size=args.max_batch_size,
    shuffle=True
)

log_dir = "runs/GovnLM-0.5B"
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

model = GovnLM(args).to(device=device, dtype=torch.bfloat16)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
sheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(dataloader))

model.train()
best_loss = float('inf')
for step, batch in tqdm(enumerate(dataloader)):
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    
    inputs = input_ids[:, :-1]  
    labels = labels[:, 1:]
    
    logits = model(inputs)
    
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(
        logits.view(-1, args.vocab_size),
        labels.reshape(-1)
    )
            
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    sheduler.step()
    
    if step % 5 == 0:
        writer.add_scalar(f"loss", loss.item(), step * args.max_batch_size * args.max_seq_len)
        writer.add_scalar(f"lr", sheduler.get_last_lr()[0], step * args.max_batch_size * args.max_seq_len)
    if loss.item() < best_loss and step > 5000:
        best_loss = loss.item()
        torch.save(model.state_dict(), f"model_best.pt")
    if step % 5000 == 0:
        torch.save(model.state_dict(), f"model_{step}.pt")

/home/karel1ne/nlp_course/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


0it [00:00, ?it/s]

512


AttributeError: 'list' object has no attribute 'to'

In [ ]:
batch_size = 16

dataloader = DataLoader(
    tokenized_dataset['train'],
    batch_size=batch_size,
    shuffle=True
)

In [4]:
from torch.utils.tensorboard import SummaryWriter
import os
log_dir = "runs/GovnLM-0.5B"
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

In [13]:
device = 'cuda'
args = GovnArgs(
    dim=1024,
    n_layers=36,
    n_heads=16,
    vocab_size=32000,
    feed_forward_hidden_dim=4096,
    norm_eps=1e-6,
    max_batch_size=16,
    max_seq_len=1024
)

In [18]:
len(dataloader)

62500

In [19]:
from datasets import load_dataset
datasets = load_dataset("glue", "mrpc")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [20]:
datasets.save_to_disk('glue-mrpc')
datasets = load_dataset('glue-mrpc')

Saving the dataset (0/1 shards):   0%|          | 0/3668 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/408 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1725 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from torch import nn
from tqdm.auto import tqdm

model = GovnLM(args).to(device=device, dtype=torch.bfloat16)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
sheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(dataloader))
model.train()
best_loss = float('inf')
for step, batch in tqdm(enumerate(dataloader)):
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    
    inputs = input_ids[:, :-1]  
    labels = labels[:, 1:]
    
    logits = model(inputs)
    
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(
        logits.view(-1, args.vocab_size),
        labels.reshape(-1)
    )
    
    perplexity = torch.exp(loss).item()
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    sheduler.step()
    
    if step % 5 == 0:
        writer.add_scalar(f"Batch{batch_size}/loss", loss.item(), step)
        writer.add_scalar(f"Batch{batch_size}/perplexity", perplexity, step)        
    if step % 200 == 0:
        torch.save(model.state_dict(), f"model_{step}.pt")
    if loss.item() < best_loss and step > 2000:
        best_loss = loss.item()
        torch.save(model.state_dict(), f"model_best.pt")

0it [00:00, ?it/s]

KeyboardInterrupt: 

: 

In [8]:
torch.save(model.state_dict(), 'model.pt')

In [4]:
model.load_state_dict(torch.load(f"model_best.pt"))

/var/tmp/ipykernel_2397210/545879141.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"model_best.pt"))


<All keys matched successfully>

In [36]:
prompt = """\
Question:
An important source of information on the credit rating of retail businesses is
Options:
(A) the Retail Merchants Association.
(B) the local chamber of commerce.
(C) Dun & Bradstreet, Inc.
(D) the United States Retail Credit Association.
Correct option:

"""

In [37]:
import torch
from tqdm.auto import tqdm

model.eval()
with torch.no_grad():
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    for _ in tqdm(range(7)):  
        logits = model(input_ids)
        next_token_logits = logits[:, -1, :]
        
        probs = torch.softmax(next_token_logits * 10, dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1)
        
        input_ids = torch.cat([input_ids, next_token_id], dim=-1)
        
        generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    print(generated_text)

  0%|          | 0/7 [00:00<?, ?it/s]

Question:
An important source of information on the credit rating of retail businesses is
Options:
(A) the Retail Merchants Association.
(B) the local chamber of commerce.
(C) Dun & Bradstreet, Inc.
(D) the United States Retail Credit Association.
Correct option:

(A) the credit rating of


In [ ]:
generated_text

'California is the richest state in America.\n\nIn a state that includes the U.S. Constitution, the state of Minnesota on the state’s second-largest federal law, has been on the rise in the past decade and has been largely associated with the state the state.\n\nThe state’s legislature has made a state-of-the-art plan to eliminate all the state’s state tax credits and state-of-the-art ordinance rules, with many states approving a $2.5-billion amendment allowing a state of the state to use a state-state solution of state-controlled state-of-the-art state laws.\n\nThat’s also the first in a state that needs more state-of-the-art legislation to change its bill for nonprofits and state governments under the age 10-21 law.\n\nBut while most county groups, it comes back and states cover big provisions seeking closer control-energy, the state-owned state package for states that have a corporate job maximise at one state level.\n\nOver the last decade, one whereby Fergus Weakuran, neither the 

: 